In [ ]:
!nvidia-smi

Sun Sep  1 06:41:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [1]:
!pip install pyarrow==14.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2


In [3]:
!pip install transformers[sentencepiece] datasets peft sacrebleu rouge_score py7zr evaluate -q

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 0.34.2
Uninstalling accelerate-0.34.2:
  Successfully uninstalled accelerate-0.34.2
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)


In [46]:
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
import evaluate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer #T5ForConditionalGeneration, PreTrainedTokenizerFast
import nltk
# from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
torch.cuda.empty_cache()

In [47]:
# Flan T5 small model from Google: 80 M parameters, 64 batch size, Dropout 0.05, Learning rate 5e-4, 98k steps
model_ckpt = "Falconsai/medical_summarization"  #"google/flan-t5-small"  # Flan T5 small
tokenizer = AutoTokenizer.from_pretrained(model_ckpt) # tokernizer maps raw textual input to number tokens.
model_T5 = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) # make predictions from the inputs.

# # Apply LoRA
# from peft import PeftModel, get_peft_model, LoraConfig
# lora_config = LoraConfig(
#     r=4,   # Rank of the adapter matrices
#     lora_alpha=16,  # Scaling factor
#     target_modules=["q", "v"],  # Targeting the query and value projection matrices of attention layers
#     lora_dropout=0.0,  # Dropout for the LoRA layers
#     bias="none"  # Do not apply LoRA to bias parameters
# )
# model_T5 = get_peft_model(model_T5, lora_config)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:  78%|#######7  | 189M/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [48]:
dataset = load_dataset("owkin/medical_knowledge_from_extracts")
dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 1383
    })
})

In [49]:
##### Splitting dataset into train, test and validation

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

# Function to extract text between 'Input:' and 'Output:'
def extract_text_between(input_string):
    try:
        # Find the start of 'Input:' and end of 'Output:'
        start = input_string.index('Input:') + len('Input:')
        end = input_string.index('Output:')
        return input_string[start:end].strip()  # Extract and trim whitespace
    except ValueError:
        return None  # Return None if 'Input:' or 'Output:' is not found

# Apply the function to the DataFrame column
df['Question'] = df['Question'].apply(extract_text_between)

# Split the data into train and temp sets (80% train, 20% temp)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split the temp set into validation and test sets (50% of temp each, i.e., 10% of the original data each)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert DataFrames to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine into a DatasetDict
split_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print(split_dataset)


DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 1106
    })
    validation: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 138
    })
    test: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 139
    })
})


In [50]:
dataset = split_dataset
split_lengths = [len(dataset[split])for split in dataset]
print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")

print("\nPrompt:")
print(dataset["test"][0]["Question"])

print("\nSummary:")
print(dataset["test"][0]["Answer"])

print("\nIndex level 0:")
print(dataset["test"][0]["__index_level_0__"])

Split lengths: [1106, 138, 139]
Features: ['Question', 'Answer', '__index_level_0__']

Prompt:
Responses to adalimumab in patients with active psoriatic arthritis who have not adequately responded to prior therapy: effectiveness and safety results from an open-label study.  To evaluate the effectiveness and safety of adalimumab in patients with active psoriatic arthritis (PsA) and an inadequate response to prior therapy. Patients were treated with subcutaneous injections of adalimumab 40 mg every other week in addition to their standard antirheumatic therapies in a 12-week, open-label study. Effectiveness evaluations at Week 12 included American College of Rheumatology (ACR) and Psoriasis Area and Severity Index (PASI) response rates, Psoriatic Arthritis Response Criteria (PsARC), active dactylitis, enthesitis, and target lesion assessment. Physical function was evaluated using the Health Assessment Questionnaire Disability Index (HAQ-DI). A total of 127 patients were enrolled. At Week

In [51]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Question'] , truncation = False) #max_length = 1024

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['Answer'], max_length = 256, truncation = False)

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [52]:
dataset_pt = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/1106 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (578 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [53]:
dataset_pt["train"]

Dataset({
    features: ['Question', 'Answer', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1106
})

In [54]:
# Training

from transformers import DataCollatorForSeq2Seq
# Data collator that will dynamically pad the inputs received, as well as the output labels, as it receives each batch.
# Each batch is padded to the maximum sequence length within that batch, rather than padding everything to a fixed length, increasing efficiency.
# It can also automatically handle the shifting of labels for teacher forcing during training.
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_T5)

In [55]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='T5_output', num_train_epochs=10, learning_rate = 5e-4,
    warmup_ratio=0.01,# Ratio of warmup steps used for a linear warmup from 0 to learning_rate.
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    weight_decay=0.01, #  The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
    logging_steps=10, # Number of update steps between two logs if logging_strategy="steps".
    eval_steps=50, #  Number of update steps between two evaluations if evaluation_strategy="steps". Defaults to the same value as logging_steps if not set.
    save_steps=100, # Number of updates steps before two checkpoint saves if save_strategy="steps".
    gradient_accumulation_steps=4, # Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
    fp16 = True,  # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    group_by_length = True, # group together samples of roughly the same length in the training dataset (to minimize padding applied and be more efficient)
)

In [56]:
# torch.cuda.empty_cache()
trainer = Trainer(model=model_T5, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["train"],
                  eval_dataset=dataset_pt["validation"])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [57]:
trainer.train()

Step,Training Loss
10,3.622500
20,2.033300
30,1.427300
40,1.480500
50,1.370600
60,1.258600
70,1.262900
80,1.117500
90,1.160600
100,1.111900


TrainOutput(global_step=690, training_loss=0.8841373498888984, metrics={'train_runtime': 402.9268, 'train_samples_per_second': 27.449, 'train_steps_per_second': 1.712, 'total_flos': 1511393089093632.0, 'train_loss': 0.8841373498888984, 'epoch': 9.963898916967509})

In [61]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device=device,
                               column_text="article", column_summary="highlights"):

    report_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    summary_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for report_batch, summary_batch in tqdm(zip(report_batches, summary_batches), total=len(report_batches)):

        inputs = tokenizer(report_batch, max_length=1024,  truncation=False, padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device),length_penalty=0.8,
                                   num_beams=8, max_length=128)
        # parameter for length penalty ensures that the model does not generate sequences that are too long.
        # Finally, we decode the generated texts,
        # replace the token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=summary_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [62]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = evaluate.load('rouge')

In [63]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'Question', column_summary= 'Answer'
)

rouge_dict = dict((rn, score[rn]) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'falconsai-finetuned'] )

100%|██████████| 5/5 [00:06<00:00,  1.40s/it]


,rouge1,rouge2,rougeL,rougeLsum
falconsai-finetuned,0.002041,0.0,0.002041,0.002041


In [64]:
## Save model
model_T5.save_pretrained("falconsai-finetuned")

In [65]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [66]:
#Load

tokenizer = PreTrainedTokenizerFast.from_pretrained("/content/tokenizer")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [67]:
#Prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset["test"][0]["Question"]
reference = dataset["test"][0]["Answer"]
pipe = pipeline("summarization", model="falconsai-finetuned",tokenizer=tokenizer)

print("Prompt:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prompt:
Responses to adalimumab in patients with active psoriatic arthritis who have not adequately responded to prior therapy: effectiveness and safety results from an open-label study.  To evaluate the effectiveness and safety of adalimumab in patients with active psoriatic arthritis (PsA) and an inadequate response to prior therapy. Patients were treated with subcutaneous injections of adalimumab 40 mg every other week in addition to their standard antirheumatic therapies in a 12-week, open-label study. Effectiveness evaluations at Week 12 included American College of Rheumatology (ACR) and Psoriasis Area and Severity Index (PASI) response rates, Psoriatic Arthritis Response Criteria (PsARC), active dactylitis, enthesitis, and target lesion assessment. Physical function was evaluated using the Health Assessment Questionnaire Disability Index (HAQ-DI). A total of 127 patients were enrolled. At Week 12, patients achieved ACR20, ACR50, and ACR70 response rates of 78.0%, 55.9%, and 21.3

In [69]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset["test"][10]["Question"]
reference = dataset["test"][10]["Answer"]
pipe = pipeline("summarization", model="falconsai-finetuned",tokenizer=tokenizer)

print("Prompt:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prompt:
Iodofiltic acid I 123 (BMIPP) fatty acid imaging improves initial diagnosis in emergency department patients with suspected acute coronary syndromes: a multicenter trial.  The aim of this study was to assess the performance of beta-methyl-p-[123I]-iodophenyl-pentadecanoic acid (BMIPP) single-photon emission computed tomography (SPECT) to detect acute coronary syndromes (ACS) in emergency department patients with chest pain. Emergency department diagnosis of chest pain is problematic, often requiring prolonged observation and stress testing. BMIPP SPECT detects abnormalities in fatty acid metabolism resulting from myocardial ischemia, even many hours after symptom cessation. Emergency department patients with suspected ACS were enrolled at 50 centers. Patients received 5 mCi BMIPP within 30 h of symptom cessation. BMIPP SPECT images were interpreted semiquantitatively by 3 blinded readers. Initial clinical diagnosis was based on symptoms, initial electrocardiograms, and troponin

In [72]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 256}
sample_text = dataset["test"][5]["Question"]
reference = dataset["test"][5]["Answer"]
pipe = pipeline("summarization", model="falconsai-finetuned",tokenizer=tokenizer)

print("Prompt:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prompt:
Attachment-based family therapy for adolescents with suicidal ideation: a randomized controlled trial.  To evaluate whether Attachment-Based Family Therapy (ABFT) is more effective than Enhanced Usual Care (EUC) for reducing suicidal ideation and depressive symptoms in adolescents. This was a randomized controlled trial of suicidal adolescents between the ages of 12 and 17, identified in primary care and emergency departments. Of 341 adolescents screened, 66 (70% African American) entered the study for 3 months of treatment. Assessment occurred at baseline, 6 weeks, 12 weeks, and 24 weeks. ABFT consisted of individual and family meetings, and EUC consisted of a facilitated referral to other providers. All participants received weekly monitoring and access to a 24-hour crisis phone. Trajectory of change and clinical recovery were measured for suicidal ideation and depressive symptoms. Using intent to treat, patients in ABFT demonstrated significantly greater rates of change on s